# Read KraKen Bitcoin Data: CSV and API 

In [43]:
import pandas as pd
import json
from datetime import datetime,date, time
import requests

In [34]:
#pd.show_versions()
print(pd.__version__)

1.2.4


## 1. Kraken CSV Historical OHLCVT 

In [35]:
bitcoin_raw_file_path = "data/XBT_OHLCVT/XBTUSD_1440.csv"

In [36]:
column_names = ["date","open", "high", "low", "close", "volume", "trades"]

In [37]:
bitcoin_pd_df = pd.read_csv(bitcoin_raw_file_path, index_col = None ,names=column_names )

In [38]:
bitcoin_pd_df.head(2)

,date,open,high,low,close,volume,trades
0,1381017600,122.00,122.00,122.00,122.00,0.1,1
1,1381104000,123.61,123.61,123.61,123.61,0.1,1


In [39]:
bitcoin_pd_df.tail(2)

,date,open,high,low,close,volume,trades
3176,1656460800,20252.5,20397.0,19828.7,20086.2,4367.319083,26340
3177,1656547200,20094.0,20139.8,18617.5,19949.9,5391.527464,32136


In [49]:
bitcoin_pd_df["date"] = pd.to_datetime(bitcoin_pd_df["date"],unit='s')

In [54]:
bitcoin_pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3178 entries, 0 to 3177
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    3178 non-null   object 
 1   open    3178 non-null   float64
 2   high    3178 non-null   float64
 3   low     3178 non-null   float64
 4   close   3178 non-null   float64
 5   volume  3178 non-null   float64
 6   trades  3178 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 173.9+ KB


In [51]:
bitcoin_pd_df["date"] = pd.to_datetime(bitcoin_pd_df["date"],unit='s').dt.date

In [52]:
bitcoin_pd_df.head(3)

,date,open,high,low,close,volume,trades
0,2013-10-06,122.00,122.00,122.00,122.00,0.1000,1
1,2013-10-07,123.61,123.61,123.61,123.61,0.1000,1
2,2013-10-08,123.91,124.19,123.90,124.18,3.9916,4


In [53]:
bitcoin_pd_df.tail(3)    

,date,open,high,low,close,volume,trades
3175,2022-06-28,20716.3,21188.5,20165.3,20251.6,3250.533589,23648
3176,2022-06-29,20252.5,20397.0,19828.7,20086.2,4367.319083,26340
3177,2022-06-30,20094.0,20139.8,18617.5,19949.9,5391.527464,32136


## 2. Kraken API Market data OHLC endpoint
https://docs.kraken.com/rest/#operation/getTickerInformation

In [11]:
kraken_ohlc_cols = ["date","open", "high", "low", "close", "vwap","volume", "trades"]
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1440&since=unix_now').json()
xbt_till_today_df = pd.DataFrame(resp["result"]['XXBTZUSD'])
xbt_till_today_df.columns = kraken_ohlc_cols
xbt_till_today_df["date"] = pd.to_datetime(xbt_till_today_df["date"],unit='s')
xbt_till_today_df["date"] = xbt_till_today_df["date"].dt.date

In [12]:
xbt_till_today_df.head(3)

,date,open,high,low,close,vwap,volume,trades
0,2020-10-21,11926.7,13241.4,11905.7,12813.0,12603.2,11497.59347437,38325
1,2020-10-22,12813.0,13199.0,12690.5,12984.1,12968.6,5337.43662325,22865
2,2020-10-23,12985.4,13032.8,12735.0,12939.0,12911.7,3897.16955007,16226


In [13]:
xbt_till_today_df.tail(3)

,date,open,high,low,close,vwap,volume,trades
717,2022-10-08,19532.8,19615.0,19249.7,19417.5,19471.7,1337.85077668,10018
718,2022-10-09,19420.0,19559.5,19335.0,19443.5,19470.7,903.92350217,9334
719,2022-10-10,19443.5,19520.0,19108.3,19237.8,19308.8,2348.44749395,12890


In [14]:
xbt_api_no_vwap = xbt_till_today_df.drop("vwap", axis=1)

In [15]:
xbt_api_no_vwap.head(2)

,date,open,high,low,close,volume,trades
0,2020-10-21,11926.7,13241.4,11905.7,12813.0,11497.59347437,38325
1,2020-10-22,12813.0,13199.0,12690.5,12984.1,5337.43662325,22865


In [16]:
xbt_api_no_vwap.tail(2)

,date,open,high,low,close,volume,trades
718,2022-10-09,19420.0,19559.5,19335.0,19443.5,903.92350217,9334
719,2022-10-10,19443.5,19520.0,19108.3,19237.8,2348.44749395,12890


## 3. Historical + Current Price

In [17]:
june30_filter = (datetime.strptime("2022-06-30", '%Y-%m-%d')).date()

In [18]:
 xbt_api_after_June30 = xbt_api_no_vwap[xbt_api_no_vwap["date"] > june30_filter]

In [19]:
 xbt_api_after_June30.head(3)

,date,open,high,low,close,volume,trades
618,2022-07-01,19949.8,20878.3,18950.0,19245.3,5953.60277338,32011
619,2022-07-02,19250.8,19430.4,18969.6,19226.9,1293.38922751,15723
620,2022-07-03,19226.9,19684.3,18738.6,19293.5,2226.88548164,17685


In [20]:
 xbt_api_after_June30.tail(3)

,date,open,high,low,close,volume,trades
717,2022-10-08,19532.8,19615.0,19249.7,19417.5,1337.85077668,10018
718,2022-10-09,19420.0,19559.5,19335.0,19443.5,903.92350217,9334
719,2022-10-10,19443.5,19520.0,19108.3,19237.8,2348.44749395,12890


#### Merge 2 DataFrames

In [55]:
bitcoin_price_all = pd.concat([bitcoin_pd_df,  xbt_api_after_June30], ignore_index=True)

In [56]:
len(bitcoin_price_all)

3280

In [57]:
bitcoin_price_all.head(3)

,date,open,high,low,close,volume,trades
0,2013-10-06,122.0,122.0,122.0,122.0,0.1,1
1,2013-10-07,123.61,123.61,123.61,123.61,0.1,1
2,2013-10-08,123.91,124.19,123.9,124.18,3.9916,4


In [58]:
bitcoin_price_all.tail(3)

,date,open,high,low,close,volume,trades
3277,2022-10-08,19532.8,19615.0,19249.7,19417.5,1337.85077668,10018
3278,2022-10-09,19420.0,19559.5,19335.0,19443.5,903.92350217,9334
3279,2022-10-10,19443.5,19520.0,19108.3,19237.8,2348.44749395,12890


In [59]:
bitcoin_price_all = bitcoin_price_all.to_csv("data/bitcoin_price_all.csv", index=False)